In [ ]:
import sys

import pandas as pd

plot_data_dbscan = pd.read_csv("plotData_bert_dbscan.csv")

sys.path.insert(1, '../utils/')

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import plotly.io
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import random
init_notebook_mode(connected=True)

r = lambda: random.randint(0,255)

traces = []
for c in plot_data_dbscan.db_cluster.unique():
    if c== -1:
        trace = go.Scatter(
            x=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-X"],
            y=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-Y"],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='grey'),
            text = plot_data_dbscan.index,
            showlegend = True,
        )
    else:
   # Create a trace
        trace = go.Scatter(
            x=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-X"],
            y=plot_data_dbscan[plot_data_dbscan["db_cluster"]==c]["t-SNE-Y"],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r()),opacity=0.2),
            text = plot_data_dbscan.db_cluster,
            showlegend = True,
        )
    traces.append(trace)

data = traces

# Plot and embed in ipython notebook
sys.path.insert(1, '../bert-tfidf/plots/')
iplot(data, filename='dbscan_context_10',image='png', image_height = 500, image_width = 700) 

## Lasso selection with printing elements in a qgrid table

In [ ]:
DISPLAYED_TABLE_COLUMNS = ["left_token_str", "right_token_str", "del_tokens_str", "ins_tokens_str", "db_cluster"]
#plot_data_dbscan_copy = plot_data_dbscan.copy()
#plot_data_dbscan = plot_data_dbscan_copy.sample(100).reset_index()

In [ ]:
import time
import qgrid
qgrid.set_grid_option('maxVisibleRows', 5)

import pdb

f = go.FigureWidget()
f.layout.dragmode = 'lasso'

scatter = f.add_scatter(x = plot_data_dbscan["t-SNE-X"], y = plot_data_dbscan["t-SNE-Y"], mode = 'markers', 
                       marker=go.scatter.Marker(size=0, opacity=0),showlegend = False)


r = lambda: random.randint(0,255)

# Create a table FigureWidget that updates on selection from points in the scatter plot of f
t = go.FigureWidget([go.Table(  
    header=dict(values=DISPLAYED_TABLE_COLUMNS,             
                
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    
    cells=dict(values=[plot_data_dbscan[col] for col in DISPLAYED_TABLE_COLUMNS],              
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5
               ))])

#counter = 0
def selection_fn(trace,points,selector):
    global sel_points, counter
    sel_points = points
#     counter += 1
    
    
    #with out:
        #clear_output()
        #display("Finished: " + str(len(points.point_inds)) + "points selected")
    #    display(qgrid.show_grid(plot_data_dbscan.loc[points.point_inds, DISPLAYED_TABLE_COLUMNS]))
    
    t.data[0].cells.values = [plot_data_dbscan.loc[points.point_inds][col] for col in DISPLAYED_TABLE_COLUMNS]

traces = []
for c in plot_data_dbscan["db_cluster"].unique():
    # Create a trace
    trace = go.Scatter(
        x=plot_data_dbscan[plot_data_dbscan["db_cluster"] == c]["t-SNE-X"],
        y=plot_data_dbscan[plot_data_dbscan["db_cluster"] == c]["t-SNE-Y"],
        mode = 'markers',
        name = str(c),
        uid = str(c),
        marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
        text = plot_data_dbscan.db_cluster,
        
        showlegend = True, 

    )
    
    f.add_trace(trace)
#trace.on_selection(selection_fn)

from ipywidgets import VBox    
scatter.on_selection(selection_fn)
VBox((f, t))
#from IPython.display import display, clear_output
#display(f)

#from ipywidgets import widgets, Output
#out = Output()
#display(out)

## Nationality

In [ ]:
r = lambda: random.randint(0,255)

traces = []

    
for c in plot_data_dbscan.db_cluster.unique():

    if c!=-1:
   # Create a trace
        trace1 = go.Scatter(
            x=plot_data_dbscan.loc[(plot_data_dbscan["db_cluster"]==c) & (plot_data_dbscan["nationality"].isna()),"t-SNE-X"],
            y=plot_data_dbscan.loc[(plot_data_dbscan["db_cluster"]==c) & (plot_data_dbscan["nationality"].isna()),"t-SNE-Y"],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r()),opacity=0.1),
            text = plot_data_dbscan.db_cluster,
            showlegend = True
        )
    traces.append(trace1)
    if c !=1:
        trace2 = go.Scatter(
            x=plot_data_dbscan.loc[(plot_data_dbscan["nationality"]=='Y') | (plot_data_dbscan["nationality"]=='N'),"t-SNE-X"],
            y=plot_data_dbscan.loc[(plot_data_dbscan["nationality"]=='Y') | (plot_data_dbscan["nationality"]=='N'),"t-SNE-Y"],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
            text = plot_data_dbscan.db_cluster,
            showlegend = True,
        )
    traces.append(trace2)

data = traces

# Plot and embed in ipython notebook
iplot(data, filename='basic-scatter')

## Cluster size frequency

In [ ]:
fig = plt.figure(figsize=(16, 10), dpi= 80, facecolor='w', edgecolor='k')
plt.xlabel('Cluster')
plt.ylabel('Size of a cluster')
plot_data_dbscan['db_cluster'].value_counts().nlargest(n=10).plot(kind='bar')

## Token frequency in left and right context

In [ ]:
import operator
import numpy as np
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'medium',
         'axes.titlesize':'medium',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
pylab.rcParams.update(params)

top = 20

def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(w) for w in wordlist if not w is np.nan]
    return dict(zip(wordlist,wordfreq))


right_context_freq = sorted(wordListToFreqDict(list(plot_data_dbscan['right_token_str'])).items(), 
                           key=operator.itemgetter(1), reverse=True)
left_context_freq = sorted(wordListToFreqDict(list(plot_data_dbscan['left_token_str'])).items(), 
                           key=operator.itemgetter(1), reverse=True)
gap_context_freq = sorted(wordListToFreqDict(list(plot_data_dbscan['ins_tokens_str'])).items(), 
                           key=operator.itemgetter(1), reverse=True)

def plot_freq(left_context_freq, right_context_freq, gap_context_freq, top):
    fig, axs = plt.subplots(nrows=3, ncols=1,figsize=(20, 30))
        
    axs[0].barh([x for x,_ in left_context_freq],[x for _,x in left_context_freq])
    axs[0].set_title(" frequency plot of top {} tokens in left context".format(top))
    axs[0].set_xlabel("frequency")
    axs[0].set_ylabel("unique tokens in left context ")
    
    axs[1].barh([x for x,_ in gap_context_freq],[x for _,x in gap_context_freq])
    axs[1].set_title(" frequency plot of top {} tokens in gap".format(top))
    axs[1].set_xlabel("frequency")
    axs[1].set_ylabel("unique tokens in gap ")
    
    axs[2].barh([x for x,_ in right_context_freq],[x for _,x in right_context_freq])
    axs[2].set_title(" frequency plot of top {} tokens in right context".format(top))
    axs[2].set_xlabel("frequency")
    axs[2].set_ylabel("unique tokens in right context ")
    return fig


fig = plot_freq(left_context_freq[0:top], right_context_freq[0:top], gap_context_freq[0:top], top)